# Texture Index

The texture index (TI) is given by the standard deviation of a unit's response to a varity of images with different textures, divided by the standard deviation of its response to shapes.

This is plotted against FOI for different networks.

In [48]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import handytools.navigator as nav
import handytools.visualizer as vis
from spectools.stimulus.wyeth import get_stimulus
from spectools.responses import get_response_wrapper, get_drr_wrapper
from spectools.metrics.metrics import responsive
from spectools.models.models import AN_layer, VGG16b_layer, ResNet18_layer

In [50]:
def get_prs(R_fills, R_outlines, hkeys, verbose=False):
    prs = {}
    respdic = {}
    for hkey in hkeys:
        R_fill = R_fills[hkey]
        R_outline = R_outlines[hkey]

        prs[hkey] = np.array([])
        respdic[hkey] = []
        
        for unit in range(R_fill.shape[0]):
            resp = responsive(R_fill[unit]) and responsive(R_outline[unit])
            if resp:
                pr = np.corrcoef(R_fill[unit], R_outline[unit])[0][1]
                prs[hkey] = np.append(prs[hkey], pr)
                respdic[hkey].append(int(unit))
            else:
                prs[hkey] = np.append(prs[hkey], np.nan)
                if verbose: print(f"Layer {hkey}, unit {unit} not responsive!")
    return prs, respdic

def linear_fit(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    y_pred = slope * x + intercept
    return x, y_pred

## AlexNet

In [51]:
# params
hkeys = list(AN_layer.keys())[:-3]
mtype = "AN"

In [52]:
TI = {}
for hkey in hkeys:
    Rc = nav.npload(nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_TK_norm=standard.npy") # originally TKRc
    TI[hkey] = []

    for Runit in Rc:
        Rshape = Runit[:225]
        Rtexture = Runit[225:]
        TI[hkey].append(np.std(Rtexture)/np.std(Rshape))

In [53]:
hkey = 6
Rc = nav.npload(nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_TK_norm=standard.npy")
idx_ti = np.argsort(TI[hkey])

fig = plt.figure(figsize=(5*3, 2*3))
for i in range(5):
    ax = fig.add_subplot(2, 5, i+1)
    Runit = Rc[idx_ti[i]]
    Rshape = Runit[:225]
    Rtexture = Runit[225:]
    ax.plot(range(225), Rshape, "k-")
    ax.plot(range(225, 393), Rtexture, "b-")
    title = round(TI[hkey][idx_ti[i]], 2)
    ax.set_title("TI = " + str(title))

for j in range(1, 6):
    ax = fig.add_subplot(2, 5, i+j+1)
    Runit = Rc[idx_ti[-j]]
    Rshape = Runit[:225]
    Rtexture = Runit[225:]
    ax.plot(range(225), Rshape, "k-")
    ax.plot(range(225, 393), Rtexture, "b-")
    title = round(TI[hkey][idx_ti[-j]], 2)
    ax.set_title("TI = " + str(title))

plt.suptitle(f"Shape-Texture Index of AlexNet, layer {AN_layer[hkey]}")
vis.common_label(fig, "figure index", "response")
plt.tight_layout()
vis.savefig("ti_AN_example.png", folders=["summary_230526"])

<Figure size 1500x600 with 0 Axes>

In [54]:
# get foi response
xn, sz, lw, fg, bg = 227, 50, 1.5, 1.0, 0.0
fill = get_stimulus(1, xn=xn, sz=sz, lw=lw, fg=fg, bg=bg)
fname = lambda hkey: f"hkey={hkey}_fill=1_xn={xn}_sz={sz}_lw={lw}_fg={fg}_bg={bg}.npy"
R_fills = get_response_wrapper(hkeys, fill, fname, mtype=mtype)
outline = get_stimulus(0, xn=xn, sz=sz, lw=lw, fg=fg, bg=bg)
fname = lambda hkey: f"hkey={hkey}_fill=0_xn={xn}_sz={sz}_lw={lw}_fg={fg}_bg={bg}.npy"
R_outlines = get_response_wrapper(hkeys, outline, fname, mtype=mtype)
prs_an_foi, resp = get_prs(R_fills, R_outlines, hkeys)

In [55]:
fig = plt.figure(figsize=(3*3, 2*3))

pos = ["lr", "lr", "ll", "ll", "lr"]
for i in range(len(hkeys)):
    hkey = hkeys[i]
    foi = prs_an_foi[hkey][resp[hkey]]
    ti = np.array(TI[hkey])[resp[hkey]]
    x, y = np.log(ti), np.arctanh(foi)
    pr = np.corrcoef(x, y)[0][1]
    coors = vis.get_location(pos[i], [min(x), max(x)], [min(y), max(y)])
    
    ax = fig.add_subplot(2, 3, i+1)
    ax.scatter(x, y, color="k", s=10)
    ax.plot(*linear_fit(x, y), color="b", linestyle="--")
    ax.set_title(AN_layer[hkey])
    vis.add_textbox(f"\u03C1 = {round(pr, 2)}", *coors, ax=ax, alpha=0.5)

vis.common_label(fig, "log(TI)", "arctanh(FOI)")
plt.suptitle("FOI v TI for AlexNet")
plt.tight_layout()
vis.savefig("ti_AN.png", folders=["summary_230526"])

<Figure size 900x600 with 0 Axes>

In [56]:
# Find specific unit

hkey = 6
foi = prs_an_foi[hkey][resp[hkey]]
ti = np.array(TI[hkey])[resp[hkey]]

units = np.where((foi > 0.8) & (ti < 1))[0]
print(units)
nav.npsave(units, nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_unitsTiMin.npy")

units = np.where((foi < 0.3))[0]
print(units)
nav.npsave(units, nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_unitsTiMax.npy")

[57 58 66 69 87]
[15 21 79]


## VGG16

In [57]:
# params
hkeys = list(VGG16b_layer.keys())[:-3]
mtype = "VGG16b"

In [58]:
TI = {}
func = np.std
for hkey in hkeys:
    Rc = nav.npload(nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_TK_norm=standard.npy")
    TI[hkey] = []

    for Runit in Rc:
        Rshape = Runit[:225]
        Rtexture = Runit[225:]
        TI[hkey].append(func(Rtexture)/func(Rshape))

In [59]:
hkey = 19
Rc = nav.npload(nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_TK_norm=standard.npy")
idx_ti = np.argsort(TI[hkey])

fig = plt.figure(figsize=(5*3, 2*3))
for i in range(5):
    ax = fig.add_subplot(2, 5, i+1)
    Runit = Rc[idx_ti[i]]
    Rshape = Runit[:225]
    Rtexture = Runit[225:]
    ax.plot(range(225), Rshape, "k-")
    ax.plot(range(225, 393), Rtexture, "b-")
    title = round(TI[hkey][idx_ti[-j]], 2)
    ax.set_title("TI = " + str(title))

j = 1
count = 1
while count <= 5:
    Runit = Rc[idx_ti[-j]]

    if np.mean(Runit) != 0:
        Rshape = Runit[:225]
        Rtexture = Runit[225:]
        ax = fig.add_subplot(2, 5, i+count+1)
        ax.plot(range(225), Rshape, "k-")
        ax.plot(range(225, 393), Rtexture, "b-")
        title = round(TI[hkey][idx_ti[i+count+1]], 2)
        ax.set_title("TI = " + str(title))
        count += 1

    j += 1

vis.common_label(fig, "Texture index", "FOI")
plt.suptitle(f"Shape-Texture Index of VGG16, layer {VGG16b_layer[hkey]}")
plt.tight_layout()
vis.savefig("ti_VGG16b_example.png", folders=["summary_230526"])

<Figure size 1500x600 with 0 Axes>

In [60]:
# get foi response
xn, sz, lw, fg, bg = 227, 50, 1.5, 1.0, 0.0
fill = get_stimulus(1, xn=xn, sz=sz, lw=lw, fg=fg, bg=bg)
fname = lambda hkey: f"hkey={hkey}_fill=1_xn={xn}_sz={sz}_lw={lw}_fg={fg}_bg={bg}.npy"
R_fills = get_response_wrapper(hkeys, fill, fname, mtype=mtype)
outline = get_stimulus(0, xn=xn, sz=sz, lw=lw, fg=fg, bg=bg)
fname = lambda hkey: f"hkey={hkey}_fill=0_xn={xn}_sz={sz}_lw={lw}_fg={fg}_bg={bg}.npy"
R_outlines = get_response_wrapper(hkeys, outline, fname, mtype=mtype)
prs_vgg16_foi, resp = get_prs(R_fills, R_outlines, hkeys)

In [61]:
fig = plt.figure(figsize=(3*3, 5*3))

pos = ["ul", "lr", "lr", "ll", "ur", "ll"] + ["ll"]*13
for i in range(len(hkeys)):
    hkey = hkeys[i]
    foi = prs_vgg16_foi[hkey][resp[hkey]]
    ti = np.array(TI[hkey])[resp[hkey]]
    x, y = np.log(ti), np.arctanh(foi)
    pr = np.corrcoef(x, y)[0][1]
    coors = vis.get_location(pos[i], [min(x), max(x)], [min(y), max(y)])
    
    ax = fig.add_subplot(5, 3, i+1)
    ax.scatter(x, y, color="k", s=10)
    ax.plot(*linear_fit(x, y), color="b", linestyle="--")
    ax.set_title(VGG16b_layer[hkey])
    vis.add_textbox(f"\u03C1 = {round(pr, 2)}", *coors, ax=ax, alpha=0.5)

vis.common_label(fig, "log(TI)", "arctanh(FOI)")
plt.suptitle("FOI v TI for VGG16")
plt.tight_layout()
vis.savefig("ti_VGG16b.png", folders=["summary_230526"])

<Figure size 900x1500 with 0 Axes>

In [62]:
# Find specific unit

hkey = hkeys[8] # conv9, =19
foi = prs_vgg16_foi[hkey][resp[hkey]]
ti = np.array(TI[hkey])[resp[hkey]]

units = np.where((foi > 0.8) & (ti < 1))[0]
print(units)
nav.npsave(units, nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_unitsTiMin.npy")

units = np.where((foi < 0.55))[0]
print(units)
nav.npsave(units, nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_unitsTiMax.npy")

[ 12  17  20  38  41  47  53  66 106 109 145 169 214 245 254 268]
[126 132 162]


## ResNet18

In [63]:
# params
hkeys = list(ResNet18_layer.keys())
mtype = "ResNet18"

In [64]:
TI = {}
for hkey in hkeys:
    Rc = nav.npload(nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_TK_norm=standard.npy")
    TI[hkey] = []

    for Runit in Rc:
        Rshape = Runit[:225]
        Rtexture = Runit[225:]
        TI[hkey].append(np.std(Rtexture)/np.std(Rshape))

In [65]:
hkey = 8
Rc = nav.npload(nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_TK_norm=standard.npy")
idx_ti = np.argsort(TI[hkey])

fig = plt.figure(figsize=(5*3, 2*3))
for i in range(5):
    ax = fig.add_subplot(2, 5, i+1)
    Runit = Rc[idx_ti[i]]
    Rshape = Runit[:225]
    Rtexture = Runit[225:]
    ax.plot(range(225), Rshape, "k-")
    ax.plot(range(225, 393), Rtexture, "b-")
    title = round(TI[hkey][idx_ti[i]], 2)
    ax.set_title("TI = " + str(title))

j = 1
count = 1
while count <= 5:
    Runit = Rc[idx_ti[-j]]

    if np.mean(Runit) != 0:
        Rshape = Runit[:225]
        Rtexture = Runit[225:]
        ax = fig.add_subplot(2, 5, i+count+1)
        ax.plot(range(225), Rshape, "k-")
        ax.plot(range(225, 393), Rtexture, "b-")
        title = round(TI[hkey][idx_ti[i+count+1]], 2)
        ax.set_title("TI = " + str(title))
        count += 1

    j += 1

vis.common_label(fig, "Texture index", "FOI")
plt.suptitle(f"Shape-Texture Index of ResNet18, layer {ResNet18_layer[hkey]}")
plt.tight_layout()
vis.savefig("ti_ResNet18_example.png", folders=["summary_230526"])

<Figure size 1500x600 with 0 Axes>

In [66]:
# get foi response
xn, sz, lw, fg, bg = 227, 50, 1.5, 1.0, 0.0
fill = get_stimulus(1, xn=xn, sz=sz, lw=lw, fg=fg, bg=bg)
fname = lambda hkey: f"hkey={hkey}_fill=1_xn={xn}_sz={sz}_lw={lw}_fg={fg}_bg={bg}.npy"
R_fills = get_response_wrapper(hkeys, fill, fname, mtype=mtype)
outline = get_stimulus(0, xn=xn, sz=sz, lw=lw, fg=fg, bg=bg)
fname = lambda hkey: f"hkey={hkey}_fill=0_xn={xn}_sz={sz}_lw={lw}_fg={fg}_bg={bg}.npy"
R_outlines = get_response_wrapper(hkeys, outline, fname, mtype=mtype)
prs_resnet18_foi, resp = get_prs(R_fills, R_outlines, hkeys)

In [68]:
fig = plt.figure(figsize=(3*3, 3*3))

pos = ["ll"]*len(hkeys)
for i in range(len(hkeys)):
    hkey = hkeys[i]
    foi = prs_resnet18_foi[hkey][resp[hkey]]
    ti = np.array(TI[hkey])[resp[hkey]]
    x, y = np.log(ti), np.arctanh(foi)
    pr = np.corrcoef(x, y)[0][1]
    coors = vis.get_location(pos[i], [min(x), max(x)], [min(y), max(y)])
    
    ax = fig.add_subplot(3, 3, i+1)
    ax.scatter(x, y, color="k", s=10)
    ax.plot(*linear_fit(x, y), color="b", linestyle="--")
    ax.set_title(ResNet18_layer[hkey])
    vis.add_textbox(f"\u03C1 = {round(pr, 2)}", *coors, ax=ax, alpha=0.5)

vis.common_label(fig, "log(TI)", "arctanh(FOI)")
plt.suptitle("FOI v TI for ResNet18")
plt.tight_layout()
vis.savefig("ti_ResNet18.png", folders=["summary_230526"])

<Figure size 900x900 with 0 Axes>

In [21]:
# Find specific unit

hkey = hkeys[5] # bb3.1, =8
foi = prs_resnet18_foi[hkey][resp[hkey]]
ti = np.array(TI[hkey])[resp[hkey]]

units = np.where((foi > 0.8) & (ti < 1))[0]
print(units)
nav.npsave(units, nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_unitsTiMin.npy")

units = np.where((foi < 0.1))[0]
print(units)
nav.npsave(units, nav.resultpath, f"responses_{mtype}", f"hkey={hkey}_unitsTiMax.npy")

[ 12  20  37  39  41  46  48  67  85 111 115 126 131 134 145 155 158 172
 179 189]
[ 66  77 140 176]
